In [1]:
import anndata2ri
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from utils.utils import load_data

adata = load_data('baron+segerstolpe')

anndata2ri.activate()
importr('Seurat')
sce = anndata2ri.py2rpy(adata.raw.to_adata())
robjects.globalenv['sce'] = sce
robjects.r("""
as(sce, 'SingleCellExperiment')
saveRDS(sce, file = 'test_rpy2.rds')

batches_data <- as.Seurat(sce, counts='X', data=NULL)
batch_list <- SplitObject(batches_data, split.by = 'batch')

for(i in 1:length(batch_list)){
  batch_list[[i]] <- NormalizeData(object = batch_list[[i]], verbose = FALSE)
}

cell_anchors <- FindIntegrationAnchors(object.list = batch_list, anchor.features = rownames(batches_data))
batches <- IntegrateData(anchorset = cell_anchors)
""")
converted = robjects.r('to_converted <- as.SingleCellExperiment(batches)')
print(adata)
print(converted)
anndata2ri.deactivate()

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


(287,)